Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime

Load Datasets

In [2]:
past = pd.read_excel("Past data.xlsx")
future = pd.read_excel("May future.xlsx")

In [5]:
past.head()

,Date,Avail Rms,OOO Rms,Rms Sold,Rms To Sell,Occ,Arr Rms,Arrivals (Non-Guaranteed),Arr Res,SO Rms,...,Trans Rm Rev,Trans ADR,Grp Pkup Rms,Grp Rm Rev,Grp ADR,Grp Rem,Room Revenue,ADR,RevPAR,Guests
0,02/26/2023,150,57,79,14,52.666667,69,2,68,10,...,8247.52,104.398987,0,0,0.00000,0,8247.52,104.398987,54.983467,129 / 4
1,02/27/2023,150,12,105,33,70.000000,83,24,83,22,...,10166.15,123.977439,23,3197,139.00000,0,13363.15,127.268095,89.087667,136 / 1
2,02/28/2023,150,2,147,1,98.000000,94,1,93,53,...,21251.83,170.014640,22,4378,199.00000,0,25629.83,174.352585,170.865533,208 / 4
3,03/01/2023,150,31,69,50,46.000000,48,6,48,21,...,7050.23,108.465077,4,625,156.25000,0,7675.23,111.235217,51.168200,95 / 0
4,03/02/2023,150,5,125,20,83.333333,90,30,89,35,...,17837.17,191.797527,32,4299,134.34375,0,22136.17,177.089360,147.574467,172 / 3


Clean Datasets

In [3]:
# removed the day from date column
def splitdate(x):
    split = x.split(" ")
    return split[0]

In [4]:
past['Date'] = past['Date'].apply(splitdate)
future['Date'] = future['Date'].apply(splitdate)

In [6]:
past.set_index('Date',inplace=True)
future.set_index('Date',inplace=True)

In [10]:
past.drop(['Guests','OOO Rms','Avail Rms','Arrivals (Non-Guaranteed)','House Use Rms','Day Use Rms','House Use Rms','No Show Rms',],axis=1,inplace=True)
future.drop(['Guests','OOO Rms','Avail Rms','Arrivals (Non-Guaranteed)','House Use Rms','Day Use Rms','House Use Rms','No Show Rms'],axis=1,inplace=True)

In [11]:
past.head()

,Rms Sold,Rms To Sell,Occ,Arr Rms,Arr Res,SO Rms,Deprt Rms,Deprt Res,Cmp Rms,Wait List Rms,Trans Rms,Trans Rm Rev,Trans ADR,Grp Pkup Rms,Grp Rm Rev,Grp ADR,Grp Rem,Room Revenue,ADR,RevPAR
Date,,,,,,,,,,,,,,,,,,,,
02/26/2023,79,14,52.666667,69,68,10,130,129,0,0,79,8247.52,104.398987,0,0,0.00000,0,8247.52,104.398987,54.983467
02/27/2023,105,33,70.000000,83,83,22,57,56,0,0,82,10166.15,123.977439,23,3197,139.00000,0,13363.15,127.268095,89.087667
02/28/2023,147,1,98.000000,94,93,53,52,52,0,0,125,21251.83,170.014640,22,4378,199.00000,0,25629.83,174.352585,170.865533
03/01/2023,69,50,46.000000,48,48,21,126,125,1,0,65,7050.23,108.465077,4,625,156.25000,0,7675.23,111.235217,51.168200
03/02/2023,125,20,83.333333,90,89,35,34,34,0,0,93,17837.17,191.797527,32,4299,134.34375,0,22136.17,177.089360,147.574467


In [12]:
future.head()

,Rms Sold,Rms To Sell,Occ,Arr Rms,Arr Res,SO Rms,Deprt Rms,Deprt Res,Cmp Rms,Wait List Rms,Trans Rms,Trans Rm Rev,Trans ADR,Grp Pkup Rms,Grp Rm Rev,Grp ADR,Grp Rem,Room Revenue,ADR,RevPAR
Date,,,,,,,,,,,,,,,,,,,,
05/12/2023,112,37,74.666667,43,43,69,29,29,0,0,29,11586.15,399.522414,83,17687,213.096386,0,29273.15,261.367411,195.154333
05/13/2023,116,33,77.333333,13,13,103,9,9,0,0,28,11028.21,393.864643,88,18682,212.295455,0,29710.21,256.122500,198.068067
05/14/2023,16,133,10.666667,6,6,10,106,106,0,0,16,2000.84,125.052500,0,0,0.000000,0,2000.84,125.052500,13.338933
05/15/2023,32,117,21.333333,24,24,8,8,8,0,0,32,3778.36,118.073750,0,0,0.000000,0,3778.36,118.073750,25.189067
05/16/2023,50,99,33.333333,27,27,23,9,9,0,0,41,5760.03,140.488537,9,1071,119.000000,0,6831.03,136.620600,45.540200


Linear Regression

In [13]:
#split data
X = past.drop('Room Revenue',axis=1)
y = past['Room Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [18]:
y_pred = model.predict(X_test)

In [20]:
r2 = r2_score(y_test,y_pred)
print(r2)

1.0


In [23]:
future_y = future.drop('Room Revenue',axis=1)
future_predict = model.predict(future_y)

In [29]:
print(future_predict)
print(r2_score(future['Room Revenue'],future_predict))
print(future['Room Revenue'])

[29273.15 29710.21  2000.84  3778.36  6831.03  3310.42 11731.47 22408.97]
1.0
Date
05/12/2023    29273.15
05/13/2023    29710.21
05/14/2023     2000.84
05/15/2023     3778.36
05/16/2023     6831.03
05/17/2023     3310.42
05/18/2023    11731.47
05/19/2023    22408.97
Name: Room Revenue, dtype: float64


Since there are columns for trans rm rev and grp rm rev, this forces the model to use that instead to calculate rm rev

In [30]:
past = past.drop(['Trans Rm Rev','Trans ADR','Grp Rm Rev','Grp ADR','ADR','RevPAR',],axis=1)
future = future.drop(['Trans Rm Rev','Trans ADR','Grp Rm Rev','Grp ADR','ADR','RevPAR',],axis=1)

In [31]:
#split data
X = past.drop('Room Revenue',axis=1)
y = past['Room Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [33]:
y_pred = model.predict(X_test)
print(r2_score(y_test,y_pred))

0.4922777087332427


In [34]:
future_y = future.drop('Room Revenue',axis=1)
future_predict = model.predict(future_y)
print(future_predict)
print(r2_score(future['Room Revenue'],future_predict))
print(future['Room Revenue'])

[ 25656.73066731  25532.20472737   -741.86516551 -23752.0347288
 -13134.21049479  -2767.88171367 -14702.7063444   11171.34480306]
-1.1224497909084894
Date
05/12/2023    29273.15
05/13/2023    29710.21
05/14/2023     2000.84
05/15/2023     3778.36
05/16/2023     6831.03
05/17/2023     3310.42
05/18/2023    11731.47
05/19/2023    22408.97
Name: Room Revenue, dtype: float64
